In [1]:
import datetime
timer_start_time = datetime.datetime.now()
import sys
sys.path.append("Scripts")
from snapshot_structures import school_tsg, caseload_projection
import email_tally

import pandas as pd
from numpy import unique
from os import listdir
import textwrap
from io import StringIO

from docx import Document
from openpyxl import Workbook
from openpyxl.reader.excel import load_workbook
from openpyxl.styles import PatternFill, Border, Side
from openpyxl.styles.numbers import FORMAT_PERCENTAGE

In [2]:
# Create list of schools
sc_entries_df = pd.read_excel('Snapshots\\All School Snapshot Prep Flow Output.xlsx', sheet_name='Site Coordination Entries')
schools = list(sc_entries_df.School.unique())

# Create dataframe to be used in the Summary for PMs spreadsheet
pm_df = pd.DataFrame(columns=['School',
                              'Caseload Progress',
                              'Current CM Caseload',
                              'Non-CM Caseload',
                              'Target Caseload',
                              'Support Before Consent Date',
                              'Missing SEL',
                              'Missing SNA',
                              'Missing SSP', 
                              'Low Check-in Average',
                              '% Students Missing at least 1 check-in',
                              'Missing SC Entries',
                              '% in TSG',
                              '% in SEL TSG',
                              'Student Tier 1 Supports',
                              'Family Tier 1 Supports',
                              'Mismatched Attributes',
                              'Unstarted TSGs',
                             ], index=schools)


<font size="5">Add email intro below:</font>

In [3]:
# Set today's date
today_str = datetime.date.today().strftime("%m-%d-%y")
current_time = datetime.datetime.now()
current_month = current_time.strftime("%B")

# Create folder name
folder_name = 'Snapshots\\School Snapshots ' + today_str

# Create word doc
document = Document()

def generate_email_text(team_name: str, month: str) -> str:
    return textwrap.dedent(
        f'''\
        Hello {team_name} team,
        
        The {month} snapshot for your school is attached.
                
        Below is a summary for the {month} snapshot:'''
    )

In [4]:
# Create a dictionary of column names for caseload details. Note: openpyxl starts col_index at 1
workbook = load_workbook('Snapshots\\All School Snapshot Prep Flow Output.xlsx')
cld_ws = workbook['Caseload Details']
cie_ws = workbook['Check-ins']
tr1_ws = workbook['Tier 1']

cld_col_num = {}
current  = 1
for col in cld_ws.iter_cols(1, cld_ws.max_column):
    cld_col_num[col[0].value] = current
    current += 1

#Create a dictionary of column names for Check-ins. Note: openpyxl starts col_index at 1
cie_col_num = {}
current  = 1
for col in cie_ws.iter_cols(1, cie_ws.max_column):
    cie_col_num[col[0].value] = current
    current += 1
    
# Create a dictionary of column names for Tier 1. Note: openpyxl starts col_index at 1
tr1_col_num = {}
current  = 1
for col in tr1_ws.iter_cols(1, tr1_ws.max_column):
    tr1_col_num[col[0].value] = current
    current += 1
    

Apply functions and generate email doc

In [5]:
for file in listdir(folder_name):
    school_name = file[:-23]
    cld_df = pd.read_excel(folder_name + '\\' + file, sheet_name='Caseload Details')

    # Create worksheets to check highlighted cells
    wb = load_workbook(folder_name + '\\' + file)
    cld_ws = wb['Caseload Details']
    cie_ws = wb['Check-ins']  
    sce_ws = wb['Site Coordination Entries']
    tr1_ws = wb['Tier 1']
    
    try:
        tr2_ws = wb['Tier II']
    except:
        tr2_ws = None
    try:
        ssp_ws = wb['SSP Goals']
    except:
        ssp_ws = None
    try:
        atr_ws = wb['Attributes Check']
    except:
        atr_ws = None
    try:
        cont_ws = wb['Contact Info']
    except:
        cont_ws = None
    
    # Get stats for Caseload Details sheet
    total_students = (cld_ws.max_row - 1)
    wrong_ids = email_tally.tally_wrong_id(cld_ws)
    cm_switch_tally = email_tally.tally_cm_switch(cld_ws)
    missing_consent = email_tally.tally_missing_consents(cld_ws)
    missing_checkin = email_tally.tally_missing_checkins(cld_ws)
    total_cm_students = email_tally.tally_caseload_total(cld_df)
    total_non_cm_students = total_students - total_cm_students
    tsg_sel_students = email_tally.sel_tsg_students(cld_ws)[0]
    initiative_percent = email_tally.sel_tsg_students(cld_ws)[1]
    all_tsg_students = email_tally.sel_tsg_students(cld_ws)[2]
    unstarted_tsgs = email_tally.list_unstarted_tsgs(cld_ws, school_name) 
    wrong_init = email_tally.tally_wrong_tsg(cld_ws)
    sel_tally = email_tally.tally_missing_sel(cld_ws)
    sna_tally = email_tally.tally_missing_sna(cld_ws)
    ssp_tally = email_tally.tally_missing_ssp(cld_ws)                          
    missing_tsg = email_tally.tally_missing_tsg(cld_ws)
    
    # Get stats for Check-ins sheet
    missing_monthly_checkin = email_tally.tally_missing_monthly_checkin(cie_ws)
    exit_check_total = email_tally.exit_check(cie_ws)[0]
    exit_check_list = str(email_tally.exit_check(cie_ws)[1])[1:-1]
    
    #Add number of school goals to school_goal dictionary
#     tally_assp_goals(tr1_ws, school_name)
#     tally_assp_goals_from_notes(tr1_ws, school_name)
    
    # Get stats for SC Entries sheet
    missing_sc = email_tally.tally_missing_sce(sce_ws)
    
    #Get stats for Tier1 sheet
    missing_par = email_tally.tally_missing_parent_numbers(tr1_ws)
    missing_notes = email_tally.tally_missing_tr1_notes(tr1_ws)
    missing_tr1_act = email_tally.tally_tr1_activities(tr1_ws)
    incorrect_tr1_act = email_tally.tally_tr1_incorrect_activities(tr1_ws)
    
    tr1_student_supports = email_tally.tally_tr1_supports(tr1_ws)[0]
    tr1_family_supports = email_tally.tally_tr1_supports(tr1_ws)[1]

    # Get stats for Tier II sheet
    if tr2_ws != None:
        missing_activities = email_tally.tally_missing_activity(tr2_ws)
        incorrect_activities = email_tally.tally_incorrect_activity(tr2_ws)
        missing_t2_notes = email_tally.tally_missing_t2_notes(tr2_ws)
                
    # Get stats for SSP Goals sheet
    if ssp_ws != None:
        baselines = email_tally.tally_baselines_zero(ssp_ws)
        low_targets = email_tally.tally_few_targets(ssp_ws)
        no_sel_goal = email_tally.tally_no_sel_goal(ssp_ws)
    
    # Get stats for Attributes sheet
    if atr_ws != None:
        mismatches = email_tally.tally_mismatched_attributes(atr_ws)
    
    # Get stats for Contact Info sheet
    if cont_ws != None:
        missing_contact = email_tally.tally_missing_contact(cont_ws)
        school_addresses = email_tally.tally_school_as_address(cont_ws)
    
    try:
        supportb4consent = round(missing_consent/total_cm_students, 2)
    except:
        supportb4consent = 0
    try:
        percentMissing1checking = round(missing_monthly_checkin/total_cm_students, 2)
    except:
        percentMissing1checking = 0
    try:
        missing_sels = round(sel_tally/total_students, 2)
    except:
        missing_sels = 0
    try:
        missing_snas = round(sna_tally/total_cm_students, 2)
    except:
        missing_snas = 0
    try:
        missing_ssps = round(ssp_tally/total_cm_students, 2)
    except:
        missing_ssps = 0
    try:
        low_checkin_avg = round(missing_checkin/total_cm_students, 2)
    except:
        low_checkin_avg = 0
    try:
        mismatched_attributes = round(mismatches/total_cm_students, 2)
    except: 
        mismatched_attributes = 0
#     print(f"Missing SNA: {sna_tally} Total CM Students: {total_cm_students}")
    
    # Add data to PM Summary dataframe
    pm_df.loc[school_name] = pd.Series({'School': school_name,
                                        'Caseload Progress':total_cm_students/caseload_projection[school_name],
                                        'Current CM Caseload':total_cm_students,
                                        'Non-CM Caseload': total_non_cm_students,
                                        'Target Caseload':caseload_projection[school_name],
                                        'Support Before Consent Date':supportb4consent,
                                        'Missing SEL':missing_sels,
                                        'Missing SNA':missing_snas,
                                        'Missing SSP':missing_ssps,
                                        'Low Check-in Average':low_checkin_avg,
                                        '% Students Missing at least 1 check-in':percentMissing1checking,
                                        'Missing SC Entries':missing_sc,
                                        '% in TSG': round(all_tsg_students/100, 2),
                                        '% in SEL TSG':round(initiative_percent/100, 2),
                                        'Student Tier 1 Supports':tr1_student_supports,
                                        'Family Tier 1 Supports':tr1_family_supports,
                                        'Mismatched Attributes': mismatched_attributes,
                                        'Unstarted TSGs': str(unstarted_tsgs)[1:-1],
                                       })
    # Write-to-file code
    # Salutation & email body
    f = StringIO()
    f.write(generate_email_text(file[:-23], current_month))
    
    f.seek(0)
    document.add_paragraph(f.read())
    
    f = StringIO()

    #Caseload Details check
    if cld_ws['A2'].value is None:
        print('No student enrollments entered in the database', file=f)
    else:
        # Number of students with incorrect Student IDs
        if wrong_ids > 0:
            print('Number of students with incorrect IDs:', wrong_ids, file=f)
            
        # Number to be switched to non-cm
        if cm_switch_tally > 0:
            print('Number of students that need to be switched to non-case managed status:', cm_switch_tally, file=f)
            
        # Consent Form
        if missing_consent > 0:
            print('Number of missing consent forms/supports outside of consent date:', missing_consent, file=f)

        # Check for caseload where less than 20% of students are in initiative
        if initiative_percent < 20:
            print('Percent of students in an SEL TSG: ' + str(initiative_percent) + '%   ' + str(tsg_sel_students) + '/' + str(total_students), file=f)
            
        # Wrong initiatives
        if wrong_init > 0:
            print('Number of students in a misclassified TSG: ', wrong_init, file=f)

        # SEL
        if sel_tally > 0:
            print('Students with missing SEAD/DESSA assessments:', sel_tally, file=f)

        # SNA
        if sna_tally > 0:
            print('Number of missing Student Needs Assessments:', sna_tally, file=f)

        # SSP
        if ssp_tally > 0:
            print('Number of missing Student Support Plans:', ssp_tally, file=f)

        # Check-ins
        if missing_checkin > 0:
            print('Students below 1 check-in per month average: ', missing_checkin, file=f)
        
        # Missing at least 1 month of check-ins
        if missing_monthly_checkin > 0:
            print('Number of students with at least one month of no check-ins:', missing_monthly_checkin, file=f)

        # Missing TSG attendance if enrolled in Initiatives
        if missing_tsg > 0:
            print('Students tagged in a TSG initiative without any Tier 2/3 TSG activities logged:', missing_tsg, file=f)
            
        # Unstarted TSGs
        if len(unstarted_tsgs) != 0:
            print('TSGs with no students enrolled:', str(unstarted_tsgs)[1:-1], file=f)

    # Check-ins: potentially exited students
    if exit_check_total > 0:
        print(f'{exit_check_total} students have had 0 check-ins for the last 3 months. Are they still with the school?: {exit_check_list}', file=f)
            
    # Missing SC Entries
    if missing_sc > 0:
        print('Number of missing site coordination entries:', missing_sc, file=f)
    
    # Tier I sheet Notifications
    if tr1_ws['A2'].value is None:
        print('No Tier I supports entered in the database', file=f)
    else:
        # Missing activities
        if missing_tr1_act > 0:
            print('Number of Tier I supports that may need an activity added:', missing_tr1_act, file=f)
        
        # Incorrect activities
        if incorrect_tr1_act > 0:
            print('Number of Tier I TSG activities that need correcting:', incorrect_tr1_act, file=f)
            
        # Missing Parents Served
        if missing_par > 0:
            print('Number of times parents were not counted in Tier 1 Family Engagement events:', missing_par, file=f)
            
        if missing_notes > 0:
            print('Number of Tier I supports without notes:', missing_notes, file=f)
            
    # Tier II sheet notifications
    if tr2_ws != None:
        # Missing tier 2 activities
        if missing_activities > 0:
            print('Number of Tier II supports that may need an activity added:', missing_activities, file=f)
            
        # Incorrect tier 2 activities
        if incorrect_activities > 0:
            print('Number of Tier II TSG activities that need correcting:', incorrect_activities, file=f)
            
        # Missing tier 2 notes
        if missing_t2_notes > 0:
            print('Number of Tier II activities missing notes:', missing_t2_notes, file=f)
    
    # SSP Goals sheet notifications
    if ssp_ws != None:
        # Goal Baseline 0
        if '99th' in file or '107th' in file or 'Grape' in file or 'All' in file:
            if baselines > 0:
                print('Number of Other (SEL) goals with a Baseline of zero:', baselines, file=f)
        
        # Fewer than 2 goal targets
        if low_targets > 0:
            print('Number of students with fewer than two goal targets set:', low_targets, file=f)
            
        # Missing SEL goal
        if no_sel_goal > 0:
            print('Number of students missing SEL goal in SSP:', no_sel_goal, file=f)
    
    # Mismatched Attributes
    if atr_ws != None:
        if mismatches > 0:
            print('Number of attributes not matching the district data:', mismatches, file=f)
    
    # Contact Info sheet notifications
    if cont_ws != None:
        # Missing Contacts
        if missing_contact > 0:
            print('Number of Student Contacts missing information:', missing_contact, file=f)
            
        # School Address used as student address
        if school_addresses > 0:
            print('Number of students with school address as personal address:', school_addresses, file=f)

 
    print('\nPlease see the Data Entry Checklist and Data Protocols for examples and instructions on how to enter and update data in the database, as well as the updated Snapshot User Guide to understand where and why things get flagged.', file=f)
    print('\nAs always, if you have any questions please don\'t hesitate to reach out to us.', file=f)
    print('\nAll the best,', file=f)

    
    f.seek(0)
    document.add_paragraph(f.read())
    if file[:-23] != 'Webster Middle School':
        document.add_page_break()


In [6]:
document.save('Snapshots\\Email Notes ' + today_str + '.docx')

Create Snapshot summary doc for Program Managers

In [7]:
from colors import pattern_fill, Color

ivy_df = pm_df[(pm_df.School == '123 Middle School')]

bob_df = pm_df[(pm_df.School == 'ABC Elementary')]

jane_df = pm_df[(pm_df.School == 'Generic High School')]

with pd.ExcelWriter(f'Snapshots/Summary for PMs ' + today_str + '.xlsx') as writer:
    pm_df.to_excel(writer, sheet_name= 'All', index=False)
    ivy_df.to_excel(writer, sheet_name= 'Ivy', index=False)
    bob_df.to_excel(writer, sheet_name= 'Bob', index=False)
    jane_df.to_excel(writer, sheet_name= 'Jane', index=False)

pm_wb = load_workbook('Snapshots\\Summary for PMs ' + today_str + '.xlsx')

for sheet in pm_wb:
    # Format column lengths
    for column_cells in sheet.columns:
        if column_cells[0].coordinate[0] == 'A':
            sheet.column_dimensions[column_cells[0].column_letter].width = 31
        else:
            length = max(len(str(cell.value)) for cell in column_cells) + 2
            sheet.column_dimensions[column_cells[0].column_letter].width = length
            
    # Caseload Progress        
    for row in sheet.iter_rows(min_row=2, min_col=2, max_col=2):
        for cell in row:
            # Start highlighting if caseload under 100% in February
            if current_time.month >= 2 and current_time.month <=7:
                if cell.value < 1:
                    pattern_fill(cell, color=Color.YELLOW)
            cell.number_format = FORMAT_PERCENTAGE
            
    # Supports b4 consent        
    for row in sheet.iter_rows(min_row=2, min_col=6, max_col=6):
        for cell in row:
            if cell.value > 0:
                pattern_fill(cell, color=Color.YELLOW)
            cell.number_format = FORMAT_PERCENTAGE
            
    # Missing SEL, SNA, SSP    
    for row in sheet.iter_rows(min_row=2, min_col=7, max_col=9):
        for cell in row:
            # Highlight if missing sel,sna, ssp >50% in November
            if current_time.month == 11:
                if cell.value > .5:
                    pattern_fill(cell, color=Color.YELLOW)
            # Highlight if missing sel,sna, ssp >25% in December
            elif current_time.month == 12:
                if cell.value > .25:
                    pattern_fill(cell, color=Color.YELLOW)
            # Highlight if missing sel,sna, ssp >0% in January+
            elif current_time.month >= 1 and current_time.month <=7:
                if cell.value > 0:
                    pattern_fill(cell, color=Color.YELLOW)
            # Format
            cell.number_format = FORMAT_PERCENTAGE
            
    # Missing Check-in avg & at least 1 checkin        
    for row in sheet.iter_rows(min_row=2, min_col=10, max_col=11):
        for cell in row:
            # Highlight if check-ins >50% starting Oct
            if current_time.month >= 10 or (current_time.month >= 1 and current_time.month <=4):
                if cell.value >= .5:
                    pattern_fill(cell, color=Color.YELLOW)
            # Highlight if check-ins >20% starting May
            elif current_time.month >= 5 and current_time.month <=7:
                if cell.value >= .2:
                    pattern_fill(cell, color=Color.YELLOW)
            #format
            cell.number_format = FORMAT_PERCENTAGE
            
    # SC Entries            
    for row in sheet.iter_rows(min_row=2, min_col=12, max_col=12):
        for cell in row:
            if int(cell.value) > 0:
                pattern_fill(cell, color=Color.YELLOW)
                
    # % in TSG & SEL TSG            
    for row in sheet.iter_rows(min_row=2, min_col=13, max_col=14):
        for cell in row:
            # Highlight if TSG/SEL TSG <15% in November + Dec
            if current_time.month >= 11:
                if cell.value < .15:
                    pattern_fill(cell, color=Color.YELLOW)
            # Highlight if TSG/SEL TSG <20% in January+
            elif current_time.month >= 1 and current_time.month <=7:
                if cell.value < .2:
                    pattern_fill(cell, color=Color.YELLOW)
            #format
            cell.number_format = FORMAT_PERCENTAGE
            
    # Student Support #s
    for row in sheet.iter_rows(min_row=2, min_col=15, max_col=15):
        for cell in row:
            # Highlight student supports < 3 starting in December
            if current_time.month == 12 or current_time.month < 4:
                for cell in row:
                    if int(cell.value) < 3:
                        pattern_fill(cell, color=Color.YELLOW)
            # Highlight student supports < 8 starting in April
            if current_time.month >= 4 and current_time.month <=7:
                if int(cell.value) < 8:
                    pattern_fill(cell, color=Color.YELLOW)
                    
    # Family Support #s
    for row in sheet.iter_rows(min_row=2, min_col=16, max_col=16):
        for cell in row:
            # Highlight student supports < 1 starting in December
            if current_time.month == 12 or current_time.month < 4:
                for cell in row:
                    if int(cell.value) < 1:
                        pattern_fill(cell, color=Color.YELLOW)
            # Highlight student supports < 4 starting in April
            if current_time.month >= 4 and current_time.month <=7:
                if int(cell.value) < 4:
                    pattern_fill(cell, color=Color.YELLOW)
    
    # Missing TSGs
    if current_time.month <=7:
        for row in sheet.iter_rows(min_row=2, min_col=18, max_col=18):
            for cell in row:
                if cell.value is not None:
                    pattern_fill(cell, color=Color.YELLOW)
                    
    # % w/ Mismatched Attributes          
    for row in sheet.iter_rows(min_row=2, min_col=17, max_col=17):
        for cell in row:
            # Highlight if missing Attributes is over 30% after March
            if current_time.month >= 3 and current_time.month < 6:
                if cell.value > .30:
                    pattern_fill(cell, color=Color.YELLOW)
            # Highlight if missing Attributes is over 0% in June
            elif current_time.month == 6:
                if cell.value > 0:
                    pattern_fill(cell, color=Color.YELLOW)
            #format
            cell.number_format = FORMAT_PERCENTAGE

                
    pm_wb.save(f'Snapshots/Summary for PMs ' + today_str + '.xlsx')
        
    
    

In [8]:
timer_end_time = datetime.datetime.now()
print('Duration: {}'.format(timer_end_time - timer_start_time))

Duration: 0:00:02.412905
